# Instalação das bibliotecas

In [17]:
!pip install fastapi pydantic uvicorn


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# Importação de bibliotecas

In [3]:
from datetime import datetime
from typing import Optional
from uuid import uuid4
from fastapi import FastAPI, HTTPException, Request
from fastapi.responses import JSONResponse
from fastapi.testclient import TestClient
from fastapi.exceptions import RequestValidationError
from pydantic import BaseModel, Field, UUID4, field_validator
from enum import Enum
from starlette.status import HTTP_422_UNPROCESSABLE_ENTITY

# instância principal da api

In [4]:
app = FastAPI()

# Definindo classe Categoria

In [5]:
class Categoria(str, Enum):    # criação da classe com Enum para ter valores fixos
    ELETRONICO = "Eletrônico"
    VESTUARIO = "Vestuário"  
    ALIMENTO = "Alimento"  
    OUTRO = "Outro"           

# Definindo classe Produto

In [8]:
class Produto(BaseModel):  # definição da classe produto
    __produtos__ = []  # lista que funciona como "banco de dados"
 
    id: UUID4 = Field(default_factory=uuid4, description="Identificador único do produto")  # id gerado automaticamente com modelo UUID4
    nome: str = Field(..., min_length=2, description="Nome do produto")  # campo obrigatório, mínimo de 2 caracteres
    preco: float = Field(..., gt=0, description="Preço deve ser maior que zero")  # campo obrigatório, preço maior que zero
    quantidade: int = Field(..., ge=0, description="Quantidade deve ser zero ou positiva")  # quantidade obrigatória, zero ou mais
    categoria: Categoria = Field(..., description="Categoria do produto")  # campo obrigatório, deve ser uma das opções na classe Categoria
    data_cadastro: datetime = Field(default_factory=datetime.now, description="Data do cadastro")  # data é gerada automaticamente no momento atual do cadastro

# Validador personalizado

In [9]:
@field_validator("nome")  # definindo um validador personalizado para o campo "nome" (padrão Pydantic v2)
@classmethod  # indica que o método recebe a classe (cls) como primeiro parâmetro
def validar_nome(cls, value: str) -> str:  # método que valida o valor do campo "nome"
    if not value.strip():  # verifica se o nome está vazio ou apenas com espaços
        raise ValueError("O nome do produto não pode estar vazio")  # lança erro se for inválido
    return value  # retorna o valor válido do nome

# Tratamento de erros da api

In [10]:
@app.exception_handler(RequestValidationError)  # define um tratador personalizado para erros de validação (422)
async def validation_exception_handler(request: Request, exc: RequestValidationError):  # função chamada quando ocorre erro de validação
    erros_traduzidos = []  # lista que armazenará os erros traduzidos
    for e in exc.errors():  # percorre cada erro retornado pelo Pydantic
        msg = e["msg"]  # pega a mensagem padrão do erro

        # traduz mensagens mais comuns do inglês para português
        if "Input should be a valid number" in msg:  # erro ao enviar valor não numérico
            msg = "O valor deve ser um número válido"
        elif "field required" in msg:  # erro de campo obrigatório ausente
            msg = "Campo obrigatório não informado"
        elif "string_too_short" in e["type"]:  # erro de string curta demais
            msg = f"O campo '{'.'.join(str(p) for p in e['loc'])}' é muito curto"
        elif "greater_than" in e["type"]:  # erro de valor numérico que deveria ser maior
            msg = "O valor deve ser maior que zero"
        elif "greater_than_equal" in e["type"]:  # erro de valor numérico que deveria ser >=
            msg = "O valor deve ser zero ou positivo"

        erros_traduzidos.append({  # adiciona erro traduzido à lista
            "campo": e["loc"],  # local do erro (campo)
            "mensagem": msg     # mensagem traduzida
        })

    return JSONResponse(  # retorna resposta JSON personalizada
        status_code=HTTP_422_UNPROCESSABLE_ENTITY,  # define status HTTP 422
        content={"detail": erros_traduzidos},  # envia lista de erros traduzidos
    )


@app.get("/produtos", response_model=list[Produto])  # rota GET que lista todos os produtos
async def listar_produtos() -> list[Produto]:  # função que retorna a lista de produtos
    return list(Produto.__produtos__)  # retorna todos os produtos armazenados


@app.post("/produtos", response_model=Produto)  # rota POST para cadastrar um novo produto
async def cadastrar_produto(produto: Produto):  # função que recebe um produto
    if any(p.nome == produto.nome for p in Produto.__produtos__):  # verifica se já existe produto com o mesmo nome
        raise HTTPException(status_code=409, detail="Produto já cadastrado")  # retorna erro 409 se duplicado
    Produto.__produtos__.append(produto)  # adiciona produto à lista
    return produto  # retorna produto cadastrado


@app.get("/produtos/{produto_id}", response_model=Produto)  # rota GET que busca um produto pelo ID
async def buscar_produto(produto_id: UUID4) -> Produto:  # função que recebe o ID do produto
    produto = next((p for p in Produto.__produtos__ if p.id == produto_id), None)  # procura produto pelo ID
    if not produto:  # se não encontrar
        raise HTTPException(status_code=404, detail="Produto não encontrado")  # retorna erro 404
    return produto  # retorna produto encontrado

# Definindo função de teste

In [12]:
def main() -> None:  # função principal de execução dos testes
    with TestClient(app) as client:  # cria cliente de teste para simular chamadas HTTP na API

        print("Testando cadastro e duplicação...")  # informa início do teste de duplicidade
        p1 = {"nome": "Notebook", "preco": 3500.00, "quantidade": 5, "categoria": "Eletrônico"}  # produto válido
        response = client.post("/produtos", json=p1)  # envia requisição POST para cadastrar produto
        print("Sucesso:", response.status_code, response.json())  # exibe resposta de sucesso
        response = client.post("/produtos", json=p1)  # tenta cadastrar mesmo produto de novo
        print("Erro duplicado:", response.status_code, response.json())  # exibe erro 409 esperado

        print("\nTestando nome válido e inválido...")  # informa início do teste de nome
        p2 = {"nome": "Camiseta", "preco": 59.90, "quantidade": 10, "categoria": "Vestuário"}  # produto válido
        response = client.post("/produtos", json=p2)  # cadastra produto válido
        print("Sucesso:", response.status_code, response.json())  # exibe resposta de sucesso
        p2_invalido = {"nome": "", "preco": 59.90, "quantidade": 10, "categoria": "Vestuário"}  # produto com nome vazio
        response = client.post("/produtos", json=p2_invalido)  # tenta cadastrar produto inválido
        print("Nome inválido:", response.status_code, response.json())  # exibe erro 422 esperado

        print("\nTestando dados corretos e tipo errado...")  # informa início do teste de tipos
        p3 = {"nome": "Arroz 5kg", "preco": 25.00, "quantidade": 100, "categoria": "Alimento"}  # produto válido
        response = client.post("/produtos", json=p3)  # cadastra produto válido
        print("Sucesso:", response.status_code, response.json())  # exibe resposta de sucesso
        p3_invalido = {"nome": "Arroz 5kg", "preco": "não é número", "quantidade": 100, "categoria": "Alimento"}  # preco com string inválida
        response = client.post("/produtos", json=p3_invalido)  # tenta cadastrar produto inválido
        print("Tipo inválido:", response.status_code, response.json())  # exibe erro 422 traduzido

        print("\nTestando busca existente e inexistente...")  # informa início do teste de busca
        todos = client.get("/produtos").json()  # pega lista de todos os produtos cadastrados
        primeiro_id = todos[0]["id"]  # pega o ID do primeiro produto da lista
        response = client.get(f"/produtos/{primeiro_id}")  # busca produto existente pelo ID
        print("Produto encontrado:", response.status_code, response.json())  # exibe produto encontrado
        response = client.get("/produtos/00000000-0000-0000-0000-000000000000")  # busca produto com ID inexistente
        print("Produto não encontrado:", response.status_code, response.json())  # exibe erro 404 esperado


# Execução de Testes

Foram realizados 4 cenários de teste que simula na API via http, cada cenário valida o comportamento da aplicação ao cadastrar e consultar produtos, com exemplo bem-sucedido e mal-sucedido

p1 — Cadastro e duplicação

- Caso de sucesso: cadastra um produto válido (Notebook) e recebe 200 OK com os dados do produto.
- Caso de erro: tenta cadastrar o mesmo produto novamente e recebe 409 Conflict com a mensagem "Produto já cadastrado".

p2 — Nome válido e inválido

- Caso de sucesso: cadastra um produto válido (Camiseta) e recebe 200 OK.
- Caso de erro: tenta cadastrar um produto com nome vazio e recebe 422 Unprocessable Entity com a mensagem "O campo 'body.nome' é muito curto".

p3 — Dados corretos e tipo errado

- Caso de sucesso: cadastra um produto válido (Arroz 5kg) e recebe 200 OK.
- Caso de erro: envia o campo preco como string em vez de número e recebe 422 Unprocessable Entity com a mensagem "O valor deve ser um número válido".

p4 — Busca existente e inexistente

- Caso de sucesso: busca um produto já cadastrado pelo seu id e recebe 200 OK com os dados do produto.
- Caso de erro: tenta buscar um produto com UUID inválido/inexistente e recebe 422 Unprocessable Entity com a mensagem "UUID version 4 expected".

In [13]:
if __name__ == "__main__":
    main()

Testando cadastro e duplicação...
Sucesso: 200 {'id': '9ae61306-3a66-46f2-b3e8-0289cbfb6af2', 'nome': 'Notebook', 'preco': 3500.0, 'quantidade': 5, 'categoria': 'Eletrônico', 'data_cadastro': '2025-08-17T15:23:13.310889'}
Erro duplicado: 409 {'detail': 'Produto já cadastrado'}

Testando nome válido e inválido...
Sucesso: 200 {'id': '6ee6aade-169a-43a2-ab63-605a92ab9d2f', 'nome': 'Camiseta', 'preco': 59.9, 'quantidade': 10, 'categoria': 'Vestuário', 'data_cadastro': '2025-08-17T15:23:13.317645'}
Nome inválido: 422 {'detail': [{'campo': ['body', 'nome'], 'mensagem': "O campo 'body.nome' é muito curto"}]}

Testando dados corretos e tipo errado...
Sucesso: 200 {'id': 'f7d8823d-4995-4022-9855-89904ff8abbd', 'nome': 'Arroz 5kg', 'preco': 25.0, 'quantidade': 100, 'categoria': 'Alimento', 'data_cadastro': '2025-08-17T15:23:13.321474'}
Tipo inválido: 422 {'detail': [{'campo': ['body', 'preco'], 'mensagem': 'O valor deve ser um número válido'}]}

Testando busca existente e inexistente...
Produto